In [1]:
    var sparkConf = new SparkConf().setAppName("Isotonic Regression Example");
    var sc = new SparkContext(sparkConf);

var filename =  "../data/mllib/sample_isotonic_regression_data.txt";
    var data = sc.textFile(filename);


 Create label, feature, weight tuples from input data with weight set to default value 1.0.


In [2]:
    var parsedData = data.map(function (line) {
        var parts = line.split(",");
        return new Tuple(parseFloat(parts[0]), parseFloat(parts[1]), 1.0);
    });


 Split data into training (60%) and test (40%) sets.


In [3]:
    var splits = parsedData.randomSplit([0.6, 0.4], 11);
    var training = splits[0];
    var test = splits[1];


 Create isotonic regression model from training data.
 Isotonic parameter defaults to true so it is only shown for demonstration


In [4]:
    var model = new IsotonicRegression().setIsotonic(true).run(training);


 Create tuples of predicted and real labels.


In [5]:
    var predictionAndLabel = test.mapToPair(function (point, model) {
        var predictedLabel = model.predict(point[1]);
        return new Tuple(predictedLabel, point[0]);

    }, [model]);


 Calculate mean squared error between predicted and real labels.


In [6]:
    var meanSquaredError = new FloatRDD(predictionAndLabel.map(function (pl) {
        return Math.pow(pl[0] - pl[1], 2);
    })).mean();

    var result = {};
    result.meanSquaredError = meanSquaredError;
    result.model = model;
    return result;
    print("Mean Squared Error = " + result.meanSquaredError);


 Save and load model


In [7]:
    result.model.save(sc, "target/tmp/myIsotonicRegressionModel");
    var sameModel = IsotonicRegressionModel.load(sc, "target/tmp/myIsotonicRegressionModel");


In [8]:
    sc.stop();
